In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import io
import sys
import os

In [ ]:
EPOCH = 100
BATCH_SIZE = 64
LR = 0.001

In [ ]:
T = transforms.ToTensor()
train_data = datasets.CIFAR10(root='~/dataset/', train=True,transform=T,download=True)
test_data =datasets.CIFAR10(root='~/dataset/',train=False,transform=T,download=True)

In [ ]:
sys.path.append('../')
from alexnet import alexnet
from torch.utils.data import DataLoader
from torchvision.models import AlexNet_Weights
train_loader = DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
model = alexnet(weights = AlexNet_Weights.IMAGENET1K_V1)


In [ ]:
test_train = test_train.cuda()
model = model.cuda()
res = model(test_train)

In [ ]:
test_train = train_data[0][0]

In [ ]:
x = transforms.Resize([256,256])(x)

In [ ]:
test_train = transforms.Resize([256,256])(test_train)

In [ ]:
test_train = transforms.ToPILImage()(test_train)
test_train